In [7]:
#Clean up bond dataset 

import pandas as pd

# Define paths
master_file_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw_copy/raw/Master_File.csv"
bonds_file_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw_copy/raw/Bonds_Sep_2023.csv"

# Load the CSV files
master_data = pd.read_csv(master_file_path)
bonds_data = pd.read_csv(bonds_file_path)

# Merge the datasets on the common bond identifier column
merged_data = pd.merge(bonds_data, master_data, on='bond_sym_id')

# Filter out specific columns and bonds you want to look at, and add the volume quantity
desired_cols = ["bond_sym_id", "yld_pt", "trd_exctn_dt", "mtrty_dt", "buy_cmsn_rt", "sell_cmsn_rt", "entrd_vol_qt"]
# desired_bond_sym_id = ["AFL.GF", "AFL4404390", "AFL4972576"]
desired_trd_exctn_dt = ["2023-09-01", "2023-09-05","2023-09-06","2023-09-07","2023-09-08","2023-09-11", "2023-09-12"]

# Apply those specific parameters to the filtered dataset
filtered_data = merged_data[desired_cols]
# filtered_data = filtered_data[filtered_data["bond_sym_id"].isin(desired_bond_sym_id)]
filtered_data = filtered_data[filtered_data["trd_exctn_dt"].isin(desired_trd_exctn_dt)]

if filtered_data.to_csv("/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw_copy/raw/filtered_data.csv", index=False):
    print("filtered data saved in the raw_copy folder")

# Filter out rows where yld_pt is 0
filtered_data = filtered_data[filtered_data['yld_pt'] > 0]

# Group by bond symbol and trade execution date, and aggregate the data
grouped_data = filtered_data.groupby(['trd_exctn_dt', 'bond_sym_id']).agg(
    volume_weighted_yield=('yld_pt', lambda x: (x * filtered_data.loc[x.index, 'entrd_vol_qt']).sum() / filtered_data.loc[x.index, 'entrd_vol_qt'].sum()),
    total_daily_volume=('entrd_vol_qt', 'sum'),
    total_buy_commission=('buy_cmsn_rt', 'sum'),  # Sum up the buy commissions
    total_sell_commission=('sell_cmsn_rt', 'sum'),  # Sum up the sell commissions
    maturity_date=('mtrty_dt', 'first')  # Use the first maturity date (assuming it's the same for all rows)
).reset_index()


# Ignore commissions (since they're mostly zero)
grouped_data = grouped_data.drop(columns=['total_buy_commission', 'total_sell_commission'])

# Save the grouped dataset to a CSV file
output_path = '/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/all_aggregated_bond_data.csv'
grouped_data.to_csv(output_path, index=False)


/var/folders/f3/v2ckd75j2zd4t4r95xdlf_yc0000gn/T/ipykernel_18191/909766775.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  bonds_data = pd.read_csv(bonds_file_path)


In [ ]:
#Switched the format of the dataset to be able to be viewed more easier
#This is for the Treasury Yields 

import pandas as pd

# Load the Excel file (adjust the sheet name if needed)
treasury_yields_path = '/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/Treasury_Yields.xlsx'
df = pd.read_excel(treasury_yields_path)

#Checking to see dimensions of dataset 
# print(df.head())
# print("num cols: ", len(df.columns))
# print("name of cols:", df.columns.tolist())

def reshape_treasury_data(df):
    reshaped_data = pd.DataFrame()  # Empty dataframe to store results
    
    # Loop through columns in pairs: (Date, Yield)
    for i in range(0, len(df.columns), 2):
        # Extract the pair of columns (Date column and the corresponding yield column)
        date_col = df.columns[i]
        yield_col = df.columns[i + 1]
        
        # Extract the block with the correct Date and Yield column
        block = df[[date_col, yield_col]].copy()
        
        # Rename columns to 'Date' and 'Yield'
        block.columns = ['Date', 'Yield']
        
        # Add a 'Maturity' column based on the yield column name
        block['Maturity'] = yield_col
        
        # Concatenate the reshaped block to the final dataframe
        reshaped_data = pd.concat([reshaped_data, block])

    return reshaped_data

# Reshape the data
reshaped_data = reshape_treasury_data(df)

# Ensure 'Date' is in datetime format
reshaped_data['Date'] = pd.to_datetime(reshaped_data['Date'], errors='coerce')

# Drop rows where the yield value is missing
reshaped_data = reshaped_data.dropna(subset=['Yield'])

# Save the cleaned and reshaped data to a CSV file
reshaped_data.to_csv('/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/Cleaned_Treasury_Yields.csv', index=False)

In [ ]:
#Extract only certain maturity time periods within the treasury dataset

import pandas as pd 

treasury_yields_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/Cleaned_Treasury_Yields.csv"
df = pd.read_csv(treasury_yields_path)

df['Date'] = pd.to_datetime(df["Date"], errors='coerce')

df_2023 = df[df["Date"].dt.year == 2023]

df_filtered = df_2023[df_2023['Maturity'].isin(['1 Mo', '2 Mo', '6 Mo'])]

filtered_csv_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/Extracted_Maturity_Treasury_Yields.csv"

df_filtered.to_csv(filtered_csv_path, index = False)



In [ ]:
#Code to calculate volume weighted yield for treasury bonds and add it to new dataset 

import pandas as pd 

treasury_path = '/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw_copy/raw/Cleaned_Treasury_Yields.csv'

df = pd.read_csv(treasury_path)


#need to look at this in more detail 
grouped_data = filtered_data.groupby(['Date']).agg(
    volume_weighted_yield=('yld_pt', lambda x: (x * filtered_data.loc[x.index, 'entrd_vol_qt']).sum() / filtered_data.loc[x.index, 'entrd_vol_qt'].sum()),
).reset_index()

In [ ]:
#Merge the two datasets into one for the model to train on 
#Will need to check on this 
import pandas as pd 

bonds_file_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/aggregated_bond_data.csv"
treasury_yields_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/Extracted_Maturity_Treasury_Yields.csv"

bonds_df = pd.read_csv(bonds_file_path)
treasury_df = pd.read_csv(treasury_yields_path)

bonds_df["trd_exctn_dt"] = pd.to_datetime(bonds_df["trd_exctn_dt"], errors="coerce")
treasury_df["Date"] = pd.to_datetime(treasury_df["Date"], errors="coerce")

merged_df = pd.merge(bonds_df, treasury_df, how= "cross")
# merged_df.fillna(method='ffill', inplace=True)

merged_data_path = "/Users/kanakgarg/Desktop/Kanak/FinanceRL/data/raw/merged_bonds_treasury_dataset.csv"

merged_df.to_csv(merged_data_path, index=False)

